In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secret key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

The loading of the data is done in these code blocks. Each of the three streams are read into three separate tables, using the credentials from the code blocks above.

In [ ]:
streaming_pin_df = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0e90e0175553-pin') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()
 

In [ ]:
streaming_user_df = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0e90e0175553-user') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

In [ ]:
streaming_geo_df = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0e90e0175553-geo') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()


After we have the data from the stream, we cast the data column as a string and create a schema for Spark to be able to label the data in a table form. Then the table is displayed. Because each table contains different data, we create the correct schema for each table. 

In [ ]:
streaming_pin_df = streaming_pin_df.selectExpr("CAST(data as STRING) AS jsonData") 
# display(pindf) 
schema = StructType([ 
                     StructField("index", StringType(), True), 
                     StructField("unique_id", StringType(), True), 
                     StructField("title", StringType(), True), 
                     StructField("description", StringType(), True), 
                     StructField("follower_count", StringType(), True),
                     StructField("poster_name", StringType(), True), 
                     StructField("tag_list", StringType(), True), 
                     StructField("is_image_or_video", StringType(), True), 
                     StructField("image_src", StringType(), True),
                     StructField("save_location", StringType(), True), 
                     StructField("category", StringType(), True), ])

streaming_pin_df = streaming_pin_df.select(from_json('jsonData',schema) \
     .alias("data")) \
     .select("data.*")  

In [ ]:
streaming_user_df = streaming_user_df.selectExpr("CAST(data as STRING) AS jsonData")

schema = StructType([ 
                     StructField("ind", StringType(), True), 
                     StructField("first_name", StringType(), True), 
                     StructField("last_name", StringType(), True), 
                     StructField("age", StringType(), True), 
                     StructField("date_joined", StringType(), True) ])

streaming_user_df = streaming_user_df.select(from_json('jsonData',schema) \
     .alias("data")) \
     .select("data.*")    

In [ ]:
streaming_geo_df = streaming_geo_df.selectExpr("CAST(data as STRING) AS jsonData") 
 
schema = StructType([ 
                     StructField("ind", StringType(), True), 
                     StructField("timestamp", StringType(), True), 
                     StructField("latitude", StringType(), True), 
                     StructField("longitude", StringType(), True), 
                     StructField("country", StringType(), True) ])

streaming_geo_df = streaming_geo_df.select(from_json('jsonData',schema) \
     .alias("data")) \
     .select("data.*")  

Here, we begin performing the data cleaning of each table. 

In [ ]:
df_pin = streaming_pin_df.select('*')

df_pin = df_pin.withColumnRenamed('index', 'ind') 
df_pin= df_pin.withColumn( 'save_location', regexp_replace('save_location', 'Local save in ', ''))
df_pin= df_pin.withColumn('follower_count', regexp_replace('follower_count', 'k', '000'))
df_pin= df_pin.withColumn('follower_count', regexp_replace('follower_count', 'M', '000000'))
df_pin= df_pin.withColumn('follower_count', col('follower_count').cast("int"))
df_pin= df_pin.withColumn('ind', col('ind').cast("int"))
df_pin = df_pin.select([when(col(c)=="No description available Story format",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="No description available",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="Untitled",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="No Title Data Available",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="User Info Error",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select([when(col(c)=="Image src error.",None).otherwise(col(c)).alias(c) for c in df_pin.columns])
df_pin = df_pin.select('ind', 'unique_id', 'title','description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location',  'category') 

In [ ]:
df_user = streaming_user_df.select('*')
  
df_user= df_user.withColumn( 'user_name', concat( 'first_name', lit(' '),'last_name'))
df_user = df_user.drop('first_name') 
df_user = df_user.drop('last_name') 
df_user= df_user.withColumn('date_joined', col('date_joined').cast("timestamp"))
df_user= df_user.withColumn('age', col('age').cast("int"))
df_user = df_user.select('ind', 'user_name', 'age','date_joined') 

In [ ]:
df_geo = streaming_geo_df.select('*')
  
df_geo= df_geo.withColumn('coordinates', struct(df_geo.latitude, df_geo.longitude))
df_geo = df_geo.drop('latitude') 
df_geo = df_geo.drop('longitude') 
df_geo= df_geo.withColumn('timestamp', col('timestamp').cast("timestamp"))
df_geo = df_geo.select('ind', 'country', 'coordinates','timestamp') 

From this point, the cleaned tables can now be sent as a stream. 

In [ ]:
df_pin.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
  .table("0e90e0175553_pin_table")

Out[125]: <pyspark.sql.streaming.StreamingQuery at 0x7f4051ba7400>

In [ ]:
df_user.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
  .table("0e90e0175553_user_table")

Out[110]: <pyspark.sql.streaming.StreamingQuery at 0x7f40519c3d00>

In [ ]:
df_geo.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
  .table("0e90e0175553_geo_table")

Out[111]: <pyspark.sql.streaming.StreamingQuery at 0x7f40479db0a0>